## Cargar paquetes

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt 
import statsmodels.graphics.tsaplots as sgt 
import statsmodels.tsa.stattools as sts 
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
sns.set()


## Leer base de datos de Pasajeros

In [ ]:
url = 'https://raw.githubusercontent.com/Chefi/Modelos-de-Control-de-Producci-n/refs/heads/main/airline-passengers.csv'

In [ ]:
df = pd.read_csv(url, index_col="Month", parse_dates=True)
df

In [ ]:
pd.offsets.__all__

In [ ]:
pd.offsets.MonthBegin._prefix 

In [ ]:
df.index.freq = 'MS'
df.index

In [ ]:
df.columns =["Pasajeros"]

In [ ]:
s_dec_additive = seasonal_decompose(df[["Pasajeros"]], model = "additive")
fig = s_dec_additive.plot();
fig.set_size_inches((16, 9))
# Tight layout to realign things
fig.tight_layout()
plt.show()

## Suavización Exponencial Simple

In [ ]:
span = 12
alpha = 2/(span+1)

Data = df['Pasajeros'][0]
Lista =[]
Pronosticos = Data
Lista.append(Pronosticos)
print(Pronosticos)
for i in range(len(df)-1):
    Data = df['Pasajeros'][i]
    Pronosticos = Pronosticos*(1-alpha)+Data*alpha
    Lista.append(Pronosticos)
    print(Pronosticos)
df['Pronosticos'] = Lista

In [ ]:
df.tail()

In [ ]:
df['SES']=SimpleExpSmoothing(df['Pasajeros']).fit(smoothing_level=alpha,optimized=False).fittedvalues
df['SES-1']=SimpleExpSmoothing(df['Pasajeros']).fit(smoothing_level=alpha,optimized=False).fittedvalues.shift(-1)

In [ ]:
df.tail()

In [ ]:
df.head()

In [ ]:
# Calculate Mean Absolute Error (MAE)
mae3_1 = mean_absolute_error(df['Pasajeros'], df['Pronosticos'])
print(f"Mean Absolute Error (MAE): {mae3_1}")

# Calculate Mean Squared Error (MSE)
mse3_1 = mean_squared_error(df['Pasajeros'], df['Pronosticos'])
print(f"Mean Squared Error (MSE): {mse3_1}")

rmse3_1 = np.sqrt(mse3_1)
print(f"Root Mean Squared Error (RMSE): {rmse3_1}")

In [ ]:
df[['Pasajeros','SES']].iloc[:].plot(figsize=(12,6)).autoscale(axis='x',tight=True);

## Suavización Exponencial Doble

https://www.statsmodels.org/dev/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.fit.html#statsmodels.tsa.holtwinters.ExponentialSmoothing.fit

In [ ]:
df['DESadd'] = ExponentialSmoothing(df['Pasajeros'], trend = 'add').fit().fittedvalues
df.head()

In [ ]:
df[['Pasajeros','SES','DESadd' ]].iloc[:].plot(figsize=(12,6)).autoscale(axis='x',tight=True);

In [ ]:
df[['Pasajeros','SES','DESadd' ]].iloc[-50:].plot(figsize=(12,6)).autoscale(axis='x',tight=True);

## Suavización Exponencial Triple

In [ ]:
df['TESadd12'] = ExponentialSmoothing(df['Pasajeros'],trend='add',seasonal='add',seasonal_periods=12).fit().fittedvalues
df.head()

In [ ]:
df['TESmul12'] = ExponentialSmoothing(df['Pasajeros'],trend='mul',seasonal='mul',seasonal_periods=12).fit().fittedvalues
df.head()

In [ ]:
df[['Pasajeros','SES','DESadd','TESadd12', 'TESmul12' ]].iloc[:].plot(figsize=(12,6)).autoscale(axis='x',tight=True);

In [ ]:
df[['Pasajeros','SES','DESadd','TESadd12', 'TESmul12' ]].iloc[-50:].plot(figsize=(12,6)).autoscale(axis='x',tight=True);

In [ ]:
df[['Pasajeros','TESadd12', 'TESmul12' ]].iloc[:].plot(figsize=(12,6)).autoscale(axis='x',tight=True);

In [ ]:
# Calculate Mean Absolute Error (MAE)
mae3_1 = mean_absolute_error(df['Pasajeros'], df['TESadd12'])
print(f"Mean Absolute Error (MAE): {mae3_1}")

# Calculate Mean Squared Error (MSE)
mse3_1 = mean_squared_error(df['Pasajeros'], df['TESadd12'])
print(f"Mean Squared Error (MSE): {mse3_1}")

rmse3_1 = np.sqrt(mse3_1)
print(f"Root Mean Squared Error (RMSE): {rmse3_1}")

In [ ]:
model_add = ExponentialSmoothing(df['Pasajeros'],trend='add',seasonal='add',seasonal_periods=12).fit()
model_mul = ExponentialSmoothing(df['Pasajeros'],trend='mul',seasonal='mul',seasonal_periods=12).fit()

In [ ]:
forecast_add = model_add.forecast(36)
forecast_mul = model_mul.forecast(36)

In [ ]:
import matplotlib.pyplot as plt

df['Pasajeros'].plot(figsize=(12,8))
forecast_add.plot(label='Forecast additive model');
forecast_mul.plot(label='Forecast multiplicative model');
plt.legend(loc='upper left')

## Estrategia de validación del modelo

In [ ]:
train_size = int(len(df) * 0.8)  # Use 80% of the data for training
train3_1, test3_1 = df[['Pasajeros']][:train_size], df[['Pasajeros']][train_size:]

In [ ]:
test3_1

In [ ]:
model = ExponentialSmoothing(train3_1, seasonal='add', seasonal_periods=12, trend='add', damped_trend=False)
fit_model = model.fit()
forecast_values3 = fit_model.forecast(steps=len(test3_1))

In [ ]:
# Calculate Mean Absolute Error (MAE)
mae3_1 = mean_absolute_error(test3_1, forecast_values3)
print(f"Mean Absolute Error (MAE): {mae3_1}")

# Calculate Mean Squared Error (MSE)
mse3_1 = mean_squared_error(test3_1, forecast_values3)
print(f"Mean Squared Error (MSE): {mse3_1}")

rmse3_1 = np.sqrt(mse3_1)
print(f"Root Mean Squared Error (RMSE): {rmse3_1}")

In [ ]:
FF = pd.DataFrame(forecast_values3)

In [ ]:
FF

In [ ]:
MM = pd.concat([train3_1,test3_1, FF], axis =1)

In [ ]:
MM.columns = ["Train", "Test", "Forecast"]
MM

In [ ]:
MM.iloc[:].plot(figsize=(12,6)).autoscale(axis='x',tight=True);

In [ ]:
model = ExponentialSmoothing(train3_1, seasonal='mul', seasonal_periods=12, trend='mul', damped_trend=False)
fit_model = model.fit()
forecast_values3 = fit_model.forecast(steps=len(test3_1))

In [ ]:
# Calculate Mean Absolute Error (MAE)
mae3_1 = mean_absolute_error(test3_1, forecast_values3)
print(f"Mean Absolute Error (MAE): {mae3_1}")

# Calculate Mean Squared Error (MSE)
mse3_1 = mean_squared_error(test3_1, forecast_values3)
print(f"Mean Squared Error (MSE): {mse3_1}")

rmse3_1 = np.sqrt(mse3_1)
print(f"Root Mean Squared Error (RMSE): {rmse3_1}")

In [ ]:
FF = pd.DataFrame(forecast_values3)

In [ ]:
MM = pd.concat([train3_1,test3_1, FF], axis =1)
MM.columns = ["Train", "Test", "Forecast"]
MM.iloc[:].plot(figsize=(12,6)).autoscale(axis='x',tight=True);